Lead means forward <br>
Lag mean behind

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead, round,sum, max

In [0]:
product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]

product_schema = StructType([
                    StructField('product_id', IntegerType(), True),
                    StructField('product_name', StringType(), True),
                    StructField('sales_date', StringType(), True),
                    StructField('price', IntegerType(), True)
])

product_df = spark.createDataFrame(product_data, product_schema)

In [0]:
emp_data = [(1,'manish',50000,'IT','m'),
(2,'vikash',60000,'sales','m'),
(3,'raushan',70000,'marketing','m'),
(4,'mukesh',80000,'IT','m'),
(5,'priti',90000,'sales','f'),
(6,'nikita',45000,'marketing','f'),
(7,'ragini',55000,'marketing','f'),
(8,'rashi',100000,'IT','f'),
(9,'aditya',65000,'IT','m'),
(10,'rahul',50000,'marketing','m'),
(11,'rakhi',50000,'IT','f'),
(12,'akhilesh',90000,'sales','m')]

emp_schema=['id','name','salary','dept','gender']

employee_df = spark.createDataFrame(data=emp_data,schema=emp_schema)

In [0]:
product_df.show()

In [0]:
window = Window.partitionBy('product_id').orderBy(col('sales_date'))

last_month_df = product_df.withColumn('previous_month_sales',lag(col('price'),1).over(window))

last_month_df.show()

In [0]:
window = Window.partitionBy('product_id').orderBy(col('sales_date'))

next_month_df = product_df.withColumn('previous_month_sales',lead(col('price'),1).over(window))

next_month_df.show()

In [0]:
last_month_df.withColumn('percentage_loss_gain',
                         round(((col('price')-col('previous_month_sales'))/col('price'))*100,2))\
                        .show()

% of sales of each month based on last 6 months

In [0]:
total_window = Window.partitionBy('product_id')  # for total product sales
new_window = Window.partitionBy('product_id').orderBy('sales_date')  # for running total

sales_df = product_df.withColumn('total_product_sales', sum(col('price')).over(total_window))\
    .withColumn('running_total', sum(col('price')).over(new_window))\
    .withColumn('per_sales', round((col('price') / col('total_product_sales') * 100),2)).show()